# NZ COVID19 flights

### Download data

In [ ]:
!curl -s https://raw.githubusercontent.com/stevemanion/nz-covid19-travel-data/master/data/meta/262-flights.csv -o 262-flights.csv

In [ ]:
!curl -s https://raw.githubusercontent.com/stevemanion/nz-covid19-travel-data/master/data/sources/open-flights/airports.dat -o airports.dat

This data comes from Steve Manion's work on enriching the Ministry of Health's data with external sources.
https://github.com/stevemanion/nz-covid19-travel-data

From the README:
> This is a travel dataset pertaining to all Covid 19 cases in New Zealand identified by the Ministry of Health. Each case record has been enriched and extended using Open Source Intelligence (OSINT), including details such as flight schedules, cruise ship itineraries, and so forth to help identify potential points of transmission between cases.

> Such knowledge is very useful in New Zealand's outbreak, as a large portion of cases can be attributed to citizens returning from abroad, cruise ship activity, and pre-lockdown events hosting international visitors.

### Manipulate

In [2]:
import pandas as pd

In [3]:
flights = pd.read_csv('./262-flights.csv')
flights

,caseId,flight,departureDate,arrivalDate,origin,destination,tailNumber
0,1,UNK,NaN,26/02/2020,Unknown-Iran,Dubai,NaN
1,1,EK450,NaN,26/02/2020,Dubai,Denpasar,A6-EGC
2,1,EK450,NaN,26/02/2020,Denpasar,Auckland,A6-ECN
3,2,NZ283,NaN,25/02/2020,Singapore,Auckland,ZK-NZN
4,3,QR920,NaN,23/02/2020,Doha,Auckland,A7-BBG
...,...,...,...,...,...,...,...
205,249,EK448,NaN,20/03/2020,Dubai,Auckland,A6-EON
206,249,NZ433,NaN,21/03/2020,Auckland,Wellington,ZK-OXK
207,256,QR920,NaN,18/03/2020,Doha,Auckland,A7-BBA
208,256,NZ5035,NaN,19/03/2020,Auckland,New Plymouth,ZK-MVU


We want to combine these origin/destination fields with something we can plot with.


Join with OpenFlights airport database to get lat/long values of airports.

In [4]:
airports = pd.read_csv('./airports.dat', header=None)
airports.columns = ['id', 'name', 'city', 'country', 'iata', 'icao', 'latitude', 'longitude', 'altitude', 'timezone', 'dst', 'tzdb', 'type', 'source'] 
airports = airports.drop_duplicates(['city'])

unique_flights = flights[flights['flight'] != 'UNK'][['origin', 'destination']].drop_duplicates()

In [5]:
joined = unique_flights.merge(
    airports,
    left_on='origin',
    right_on='city',
    how='left'
).merge(
    airports,
    left_on='destination',
    right_on='city',
    suffixes='_2',
    how='left'
)[['origin', 'destination', 'latitude_', 'longitude_', 'latitude2', 'longitude2']]

joined['startLat'] = joined['latitude_']
joined['startLng'] = joined['longitude_']
joined['endLat'] = joined['latitude2']
joined['endLng'] = joined['longitude2']

del joined['latitude_']
del joined['longitude_']
del joined['latitude2']
del joined['longitude2']

In [6]:
joined

,origin,destination,startLat,startLng,endLat,endLng
0,Dubai,Denpasar,25.252800,55.364399,-8.748170,115.167000
1,Denpasar,Auckland,-8.748170,115.167000,-37.008099,174.792007
2,Singapore,Auckland,1.416950,103.867996,-37.008099,174.792007
3,Doha,Auckland,25.261101,51.565102,-37.008099,174.792007
4,Houston,Auckland,29.984400,-95.341400,-37.008099,174.792007
5,Auckland,Christchurch,-37.008099,174.792007,-43.489399,172.531998
6,Brisbane,Wellington,-27.570299,153.007996,-41.327202,174.804993
7,Los Angeles,Auckland,-37.401699,-72.425400,-37.008099,174.792007
8,Auckland,Wellington,-37.008099,174.792007,-41.327202,174.804993
9,Auckland,Dunedin,-37.008099,174.792007,-45.928101,170.197998


### Plotting

In [7]:
%matplotlib widget

In [15]:
import cartopy
import cartopy.crs as ccrs
import numpy as np
import matplotlib.pyplot as plt

akl = airports[airports.city == 'Auckland']
akl_coords = (float(akl.longitude), float(akl.latitude))


projections = (
    ccrs.cartopy.crs.Robinson(akl_coords[0]),
    ccrs.NearsidePerspective(*akl_coords, satellite_height=885831)
)

for index, proj in enumerate(projections):
    plt.figure()
    ax = plt.axes(projection=proj)
    ax.set_global()

    ax.add_feature(cartopy.feature.OCEAN, zorder=0)
    ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black')
    ax.add_feature(cartopy.feature.BORDERS, linestyle='-')

    crs = ccrs.Geodetic()
    for i, a, b, start_lat, start_long, end_lat, end_long in joined.itertuples():
        start_transformed = proj.transform_point(start_long, start_lat, crs)
        end_transformed = proj.transform_point(end_long, end_lat, crs)
        plt.plot(
            (start_transformed[0], end_transformed[0]),
            (start_transformed[1], end_transformed[1]),
            linewidth=0.3,
            marker='o',
            markersize=2,
            color='red',
        )

    ax.gridlines()
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …